In [99]:
import numpy as np
import pandas as pd

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    f1_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef,
    classification_report, confusion_matrix, log_loss
)
import plotly.graph_objects as go
from cassandra.cluster import Cluster
from datetime import datetime

In [100]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('data_stock')

In [101]:
rows_fin = session.execute("""
    SELECT * FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_financial = pd.DataFrame(rows_fin)
df_financial.head(10)

,symbol,aumsize,average,change,eps,exchange,exerciseprice,exerciseratio,high,impliedvolatility,...,securitytype,status,theoretical,tolasttrade,totalbuyvolume,totalnosidevolume,totalsellvolume,totalvolume,underlying,underlyingprice
0,PPPM,None,None,None,0.02000,None,None,None,None,None,...,CS,CB,None,None,0,0,0,0,None,None
1,TPCH,None,None,None,0.24112,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None
2,KPNREIT,None,None,None,NaN,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None
3,POLY,None,None,None,0.27632,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None
4,QHBREIT,None,None,None,NaN,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None
5,VCOM,None,None,None,0.21428,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None
6,KDH,None,None,None,4.12186,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None
7,NVD,None,None,None,-0.00942,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None
8,JDF,None,None,None,0.04224,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None
9,SVR,None,None,None,-0.03402,None,None,None,None,None,...,CS,,None,None,0,0,0,0,None,None


In [102]:
rows_price = session.execute("""
    SELECT * FROM candlestick_data ALLOW FILTERING
""")
df_price = pd.DataFrame(rows_price)
df_price['time'] = pd.to_datetime(df_price['time'])
df_price = df_price.sort_values(['symbol', 'time'])

In [103]:
df_price.head()

,symbol,time,close_price,high_price,low_price,open_price,value,volume
362761,24CS,2022-10-03,10.20,10.20,7.10,7.10,0.0,559474246
362762,24CS,2022-10-04,7.15,11.10,7.15,10.70,0.0,330716796
362763,24CS,2022-10-05,5.15,6.45,5.05,5.85,0.0,361105489
362764,24CS,2022-10-06,5.20,5.45,4.70,5.40,0.0,232684182
362765,24CS,2022-10-07,5.00,5.15,4.76,5.10,0.0,131780847


In [104]:
print("Total unique symbols:", df_price['symbol'].nunique())
print("Total unique symbols:", df_financial['symbol'].nunique())

Total unique symbols: 912
Total unique symbols: 913


In [105]:
def compute_emas(df, spans=[5, 15, 35, 89, 200]):
    for span in spans:
        df[f'EMA{span}'] = df.groupby('symbol')['close_price'].transform(lambda x: x.ewm(span=span, adjust=False).mean())
    return df

df_price2 = compute_emas(df_price)


In [106]:
def compute_rsi_ema(df, window=14):
    def rsi_calc(prices):
        delta = prices.diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)

        avg_gain = gain.ewm(alpha=1/window, min_periods=window).mean()
        avg_loss = loss.ewm(alpha=1/window, min_periods=window).mean()
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))

    df['RSI'] = df.groupby('symbol')['close_price'].transform(rsi_calc)
    return df

df_price2 = compute_rsi_ema(df_price)


In [107]:
import pandas as pd
import numpy as np

# ✅ ค่า band_pct สำหรับตรวจสอบ sideway zone
band_pct = 0.015  # 1.5%

# ✅ ตัวอย่าง DataFrame ที่คุณมีอยู่แล้ว (แทนด้วย df_candle)
# ต้องมีคอลัมน์: Close, RSI, EMA5, EMA15, EMA35, EMA89, EMA200

def classify_trend(row):
    # ✅ A: Overbought / แรงมาก
    if row['close_price'] >= row['EMA5'] and row['RSI'] >= 70:
        return 'a'
    
    # ✅ B: ขาขึ้นชัดเจน
    elif row['close_price'] >= row['EMA35'] and row['EMA35'] >= row['EMA89']:
        return 'b'
    
    # ✅ C: Sideway เหนือ EMA89
    elif (row['close_price'] >= row['EMA89']):
        emas = [row['EMA5'], row['EMA15'], row['EMA35'], row['EMA89']]
        band = (max(emas) - min(emas)) / np.mean(emas)
        if band <= band_pct:
            return 'c'
    
    # ✅ D: ขาลง
    elif row['close_price'] < row['EMA89'] and row['close_price'] < row['EMA200'] and row['EMA89'] < row['EMA200']:
        return 'd'
    
    # ✅ E: Crash / ร่วงหนัก
    elif (
        row['close_price'] < row['EMA5'] < row['EMA15'] < row['EMA35'] < row['EMA89'] < row['EMA200']
        and row['RSI'] <= 30
    ):
        return 'e'
    
    # ✅ Default: ไม่เข้าเงื่อนไขใดเลย
    return 'unclassified'

# ✅ Apply ฟังก์ชันกับ DataFrame
df_price2['trend_group'] = df_price2.apply(classify_trend, axis=1)


In [108]:
df_price2.head()

,symbol,time,close_price,high_price,low_price,open_price,value,volume,EMA5,EMA15,EMA35,EMA89,EMA200,RSI,trend_group
362761,24CS,2022-10-03,10.20,10.20,7.10,7.10,0.0,559474246,10.200000,10.200000,10.200000,10.200000,10.200000,NaN,b
362762,24CS,2022-10-04,7.15,11.10,7.15,10.70,0.0,330716796,9.183333,9.818750,10.030555,10.132222,10.169652,NaN,d
362763,24CS,2022-10-05,5.15,6.45,5.05,5.85,0.0,361105489,7.838889,9.235156,9.759413,10.021506,10.119705,NaN,d
362764,24CS,2022-10-06,5.20,5.45,4.70,5.40,0.0,232684182,6.959259,8.730762,9.506113,9.914361,10.070752,NaN,d
362765,24CS,2022-10-07,5.00,5.15,4.76,5.10,0.0,131780847,6.306173,8.264416,9.255773,9.805153,10.020297,NaN,d


eps,pe,pbv,percentYield,marketcap

In [109]:
# ดึงข้อมูลจาก Cassandra
rows_fin2 = session.execute("""
    SELECT symbol, eps, pe, pbv, percentYield
    FROM financal_data_fromsettradeAPI
    ALLOW FILTERING
""")

# แปลงเป็น DataFrame
df_financial2 = pd.DataFrame(rows_fin2, columns=["symbol", "eps", "pe", "pbv", "percentYield"])

# แสดง 5 แถวแรก
df_financial2.head()


,symbol,eps,pe,pbv,percentYield
0,PPPM,0.02000,0.00,0.70,0.00
1,TPCH,0.24112,7.90,0.34,13.91
2,KPNREIT,NaN,NaN,0.23,0.00
3,POLY,0.27632,12.89,2.42,6.38
4,QHBREIT,NaN,NaN,0.41,0.00


In [110]:
# ตาราง fundamental (มี pbv, pe, eps)
rows_fin = session.execute("""
    SELECT symbol, eps, pe, pbv,percentYield FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_fin = pd.DataFrame(rows_fin, columns=["symbol", "eps", "pe", "pbv","percentYield"])

# ตาราง candlestick (มี close_price ล่าสุด)
rows_price = session.execute("""
    SELECT symbol, time, close_price,volume FROM candlestick_data ALLOW FILTERING
""")
df_price = pd.DataFrame(rows_price, columns=["symbol", "time", "close_price","volume"])

In [111]:
# ✅ 2. หาราคาปิดล่าสุด (close_price ล่าสุดของแต่ละ symbol)
df_price_latest = (
    df_price.sort_values("time")
    .groupby("symbol")
    .last()
    .reset_index()
)

In [112]:
# ✅ 3. Merge ข้อมูลทั้งสองตารางเข้าด้วยกัน
df_merged = pd.merge(df_fin, df_price_latest[["symbol", "close_price","volume"]], on="symbol", how="inner")


In [113]:
df_merged["marketcap"] = df_merged.apply(
    lambda row: row["volume"] * row["close_price"]
    if all(isinstance(row[col], (int, float)) for col in ["volume", "close_price"])
    else None,
    axis=1
)

In [114]:
df_merged.head()

,symbol,eps,pe,pbv,percentYield,close_price,volume,marketcap
0,PPPM,0.02000,0.00,0.70,0.00,0.49,313650,153688.502991
1,TPCH,0.24112,7.90,0.34,13.91,3.08,52708,162340.635979
2,KPNREIT,NaN,NaN,0.23,0.00,2.56,100,255.999994
3,POLY,0.27632,12.89,2.42,6.38,6.95,5100,35444.999027
4,QHBREIT,NaN,NaN,0.41,0.00,3.44,140301,482635.448028


In [115]:
# ✅ 6. เตรียมข้อมูลสำหรับ KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans


X = df_merged[["eps", "pe", "pbv", "marketcap"]]


# ✅ 7. จัดการ NaN ด้วย SimpleImputer
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)


# ✅ 8. ทำ Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)


# ✅ 9. KMeans Clustering
kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
df_merged['group_num'] = kmeans.fit_predict(X_scaled)


# ✅ 10. Map group number เป็น A-E
group_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
df_merged['group'] = df_merged['group_num'].map(group_map)


# ✅ แสดงผลลัพธ์สุดท้าย
df_merged[["symbol", "eps", "pe", "pbv", "close_price", "volume",  "marketcap", "group"]].head()

,symbol,eps,pe,pbv,close_price,volume,marketcap,group
0,PPPM,0.02000,0.00,0.70,0.49,313650,153688.502991,A
1,TPCH,0.24112,7.90,0.34,3.08,52708,162340.635979,A
2,KPNREIT,NaN,NaN,0.23,2.56,100,255.999994,A
3,POLY,0.27632,12.89,2.42,6.95,5100,35444.999027,A
4,QHBREIT,NaN,NaN,0.41,3.44,140301,482635.448028,A


In [116]:
# ✅ 1. ดึงข้อมูล fundamental
rows_fin = session.execute("""
    SELECT symbol, eps, pe, pbv FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_fin = pd.DataFrame(rows_fin, columns=["symbol", "eps", "pe", "pbv"])

# ✅ 2. ดึงข้อมูลราคาปิดล่าสุดจาก candlestick_data
rows_price = session.execute("""
    SELECT symbol, time, close_price, volume FROM candlestick_data ALLOW FILTERING
""")
df_price = pd.DataFrame(rows_price, columns=["symbol", "time", "close_price", "volume"])

# ✅ 3. หาราคาปิดล่าสุดและ volume ล่าสุด
latest_price = df_price.sort_values("time").groupby("symbol").last().reset_index()

# ✅ 4. Merge fundamental + price info
df_merged = pd.merge(df_fin, latest_price[["symbol", "close_price", "volume"]], on="symbol", how="inner")

# ✅ 5. คำนวณ bvps และ marketcap

df_merged["bvps"] = df_merged.apply(
    lambda row: row["close_price"] / row["pbv"] if row["pbv"] not in (0, None) else None,
    axis=1
)

df_merged["marketcap"] = df_merged.apply(
    lambda row: row["close_price"] * row["volume"]
    if all(isinstance(row[col], (int, float)) for col in ["close_price", "volume"])
    else None,
    axis=1
)

# ✅ 6. เตรียมข้อมูลสำหรับ KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

X = df_merged[["eps", "pe", "pbv", "bvps", "marketcap"]]

# ✅ 7. จัดการ NaN ด้วย SimpleImputer
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)

# ✅ 8. ทำ Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# ✅ 9. KMeans Clustering
kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
df_merged['group_num'] = kmeans.fit_predict(X_scaled)

# ✅ 10. Map group number เป็น A-E
group_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
df_merged['group'] = df_merged['group_num'].map(group_map)

# ✅ แสดงผลลัพธ์สุดท้าย
df_merged[["symbol", "eps", "pe", "pbv", "close_price", "volume", "bvps", "marketcap", "group"]].head()


,symbol,eps,pe,pbv,close_price,volume,bvps,marketcap,group
0,PPPM,0.02000,0.00,0.70,0.49,313650,0.700000,153688.502991,A
1,TPCH,0.24112,7.90,0.34,3.08,52708,9.058823,162340.635979,A
2,KPNREIT,NaN,NaN,0.23,2.56,100,11.130435,255.999994,A
3,POLY,0.27632,12.89,2.42,6.95,5100,2.871901,35444.999027,A
4,QHBREIT,NaN,NaN,0.41,3.44,140301,8.390244,482635.448028,A


In [117]:
# สมมุติ df_candle = candlestick + trend_group
#         df_financial = fundamental + group

# เลือกคอลัมน์ที่จำเป็น
df1 = df_merged[['symbol', 'group']]
df2 = df_price2[['symbol', 'trend_group']]  # 'group' คือ fundamental group (A-E)

# ✅ รวม group ทั้งสองกลายเป็น quadrant label
df_merged2 = pd.merge(df1, df2, on='symbol', how='inner')
df_merged2['quadrant'] = df_merged2['group'] + df_price2['trend_group']


In [119]:
# เลือก feature จาก candlestick (เทคนิคัล)
tech_features = df_price2[['symbol', 'close_price', 'EMA5', 'EMA15', 'EMA35', 'EMA89', 'EMA200', 'RSI']]

# เลือก feature จาก fundamental
fin_features = df_merged[["symbol", "eps", "pe", "pbv", "close_price", "volume",  "marketcap", "group"]]

# ✅ รวมทั้งหมดตาม 'symbol' และ 'time'
df_all = pd.merge(df_merged, tech_features, on=['symbol'], how='inner')
df_all = pd.merge(df_all, fin_features, on='symbol', how='inner')


In [123]:
# ✅ 1. ดึงข้อมูล fundamental
rows_fin = session.execute("""
    SELECT symbol, eps, pe, pbv, percentYield FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_fin = pd.DataFrame(rows_fin, columns=["symbol", "eps", "pe", "pbv", "percentYield"])

# ✅ 2. ดึงข้อมูลราคาปิดล่าสุดและ volume จาก candlestick_data
rows_price = session.execute("""
    SELECT symbol, time, close_price, volume FROM candlestick_data ALLOW FILTERING
""")
df_price = pd.DataFrame(rows_price, columns=["symbol", "time", "close_price", "volume"])

# ✅ 3. หาราคาปิดและปริมาณล่าสุด
latest_price = df_price.sort_values("time").groupby("symbol").last().reset_index()

# ✅ 4. Merge fundamental + price info
df_merged = pd.merge(df_fin, latest_price, on="symbol", how="inner")

# ✅ 5. คำนวณ bvps และ marketcap (จาก close_price และ volume)
df_merged["bvps"] = df_merged.apply(
    lambda row: row["close_price"] / row["pbv"] if row["pbv"] not in (0, None) else None,
    axis=1
)
df_merged["market_cap"] = df_merged.apply(
    lambda row: row["close_price"] * row["volume"]
    if all(isinstance(row[col], (int, float)) for col in ["close_price", "volume"])
    else None,
    axis=1
)

# ✅ 6. เตรียมข้อมูลสำหรับ KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

features_for_clustering = ['eps', 'pe', 'pbv', 'percentYield', 'volume', 'market_cap', 'bvps']
X = df_merged[features_for_clustering]

# ✅ 7. จัดการ missing values และ normalize
X_cleaned = X.dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cleaned)

# ✅ 8. KMeans Clustering
kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
cluster_labels = kmeans.fit_predict(X_scaled)

# ✅ 9. ใส่ผลลัพธ์กลับเข้า df
df_clustered = df_merged.loc[X_cleaned.index].copy()
df_clustered['group_num'] = cluster_labels

group_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
df_clustered['group_letter'] = df_clustered['group_num'].map(group_map)

# ✅ 10. แสดงผล
df_clustered[['symbol', 'group_letter', 'group_num'] + features_for_clustering].head()


,symbol,group_letter,group_num,eps,pe,pbv,percentYield,volume,market_cap,bvps
0,PPPM,A,0,0.02000,0.00,0.70,0.00,313650,153688.502991,0.700000
1,TPCH,D,3,0.24112,7.90,0.34,13.91,52708,162340.635979,9.058823
3,POLY,D,3,0.27632,12.89,2.42,6.38,5100,35444.999027,2.871901
5,VCOM,D,3,0.21428,7.85,1.48,10.79,335100,837750.000000,1.689189
6,KDH,A,0,4.12186,10.62,1.93,3.01,1200,102000.000000,44.041451


In [144]:
rows_fin2 = session.execute("""
    SELECT symbol,eps,pe,pbv,percentYield FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_fundamental= pd.DataFrame(rows_fin2)
df_fundamental.head(10)

,symbol,eps,pe,pbv,percentyield
0,PPPM,0.02000,0.00,0.70,0.00
1,TPCH,0.24112,7.90,0.34,13.91
2,KPNREIT,NaN,NaN,0.23,0.00
3,POLY,0.27632,12.89,2.42,6.38
4,QHBREIT,NaN,NaN,0.41,0.00
5,VCOM,0.21428,7.85,1.48,10.79
6,KDH,4.12186,10.62,1.93,3.01
7,NVD,-0.00942,38.87,0.32,0.00
8,JDF,0.04224,12.19,1.35,4.85
9,SVR,-0.03402,0.00,0.34,0.00


In [145]:
rows_price = session.execute("""
    SELECT * FROM candlestick_data ALLOW FILTERING
""")
df_candlestick_latest = pd.DataFrame(rows_price)
df_candlestick_latest['time'] = pd.to_datetime(df_candlestick_latest['time'])
df_candlestick_latest = df_candlestick_latest.sort_values(['symbol', 'time'])


In [146]:
# merge fundamental & candlestick
df = pd.merge(df_fundamental, df_candlestick_latest, on="symbol")


# คำนวณ features
df["marketcap"] = df["close_price"] * df["volume"]


In [147]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

X = df[['eps', 'pe', 'pbv', 'percentyield', 'marketcap']]
X = X.dropna()
X_scaled = StandardScaler().fit_transform(X)

kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
df.loc[X.index, 'group'] = kmeans.fit_predict(X_scaled)

# Map เป็น A–E
group_map = {i: chr(65+i) for i in range(5)}
df['group'] = df['group'].map(group_map)


In [148]:
import pandas as pd
import numpy as np

# ✅ ค่า band_pct สำหรับตรวจสอบ sideway zone
band_pct = 0.015  # 1.5%

# ✅ ตัวอย่าง DataFrame ที่คุณมีอยู่แล้ว (แทนด้วย df_candle)
# ต้องมีคอลัมน์: Close, RSI, EMA5, EMA15, EMA35, EMA89, EMA200

def classify_trend(row):
    # ✅ A: Overbought / แรงมาก
    if row['close_price'] >= row['EMA5'] and row['RSI'] >= 70:
        return 'a'
    
    # ✅ B: ขาขึ้นชัดเจน
    elif row['close_price'] >= row['EMA35'] and row['EMA35'] >= row['EMA89']:
        return 'b'
    
    # ✅ C: Sideway เหนือ EMA89
    elif (row['close_price'] >= row['EMA89']):
        emas = [row['EMA5'], row['EMA15'], row['EMA35'], row['EMA89']]
        band = (max(emas) - min(emas)) / np.mean(emas)
        if band <= band_pct:
            return 'c'
    
    # ✅ D: ขาลง
    elif row['close_price'] < row['EMA89'] and row['close_price'] < row['EMA200'] and row['EMA89'] < row['EMA200']:
        return 'd'
    
    # ✅ E: Crash / ร่วงหนัก
    elif (
        row['close_price'] < row['EMA5'] < row['EMA15'] < row['EMA35'] < row['EMA89'] < row['EMA200']
        and row['RSI'] <= 30
    ):
        return 'e'
    
    # ✅ Default: ไม่เข้าเงื่อนไขใดเลย
    return 'unclassified'

# ✅ Apply ฟังก์ชันกับ DataFrame
df['trend_group'] = df.apply(classify_trend, axis=1)


KeyError: 'EMA5'

In [142]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# สร้าง label Quadrant = group + trend_group
df["quadrant"] = df["group"].astype(str) + df["trend_group"].astype(str)
df = df.dropna(subset=["quadrant"])

# เลือก features และ label
features = ['eps', 'pe', 'pbv', 'dividend_yield', 'bvps', 'marketcap', 'rsi', 'ema5', 'ema15', 'ema35', 'ema89', 'ema200']
X = df[features]
y = LabelEncoder().fit_transform(df["quadrant"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้าง MLP model
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(features),)),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Accuracy: {acc * 100:.2f}%")


KeyError: 'trend_group'